In [2]:
import logging
from pathlib import Path

import pandas as pd
from dotenv import dotenv_values

from generator_drainage_units import run_generator_drainage_units

logging.basicConfig(level=logging.INFO)
pd.set_option("future.no_silent_downcasting", True)

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
# Define case and base_dir
config = dotenv_values("..\\.env")
base_dir = Path(config["BASE_DIR"])
# case_name
# case_name = "vallei_en_veluwe"
# case_name = "geerestein"
# case_name = "hattemerbroek"
# case_name = "pangelerbeek"
case_name = "Leuvenumse_beek"
case_path = Path(base_dir, case_name)

dir_basisdata = "0_basisdata"
dir_results = "1_resultaat"
ghg_file_name = "GHG_2000-2010_L1.NC"

In [ ]:
gdu = run_generator_drainage_units(
    path=case_path,
    dir_basisdata=dir_basisdata,
    dir_results=dir_results,
    ghg_file_name=ghg_file_name,
    read_results=True,
    write_results=True,
    preprocess=True,
    process=True,
    postprocess=False,
    resolution=2.0,
    depth_waterways=0.2,
    buffer_waterways=2.5,
    smooth_distance=25.0,
    iterations=5,
    iteration_group=100,
)

In [ ]:
gdu.aggregate_drainage_units()

In [ ]:
gdu.generate_folium_map(
    order_labels=True,
    zmin=7.5,
    zmax=25.0,
    dx=0.0,
    dy=-9.0,
)

In [ ]:
import geopandas as gpd

In [ ]:
gdu.drainage_units_0_gdf

In [ ]:
gdu.all_waterways_0

In [ ]:
sjoin = gpd.sjoin(
    gdu.drainage_units_0_gdf.rename(columns={"drainage_unit_id": "drainage_unit_id_old"}),
    gdu.all_waterways_0.drop(columns="color_id"), 
    how="inner", 
    predicate="intersects"
)

In [ ]:
sjoin = sjoin.merge(
    gdu.all_waterways_0[["geometry", "drainage_unit_id"]].rename(columns={"geometry": "waterway_geometry"}), 
    how="left", 
    on="drainage_unit_id", 
)

In [ ]:
sjoin['overlap_length'] = sjoin.geometry.intersection(sjoin.waterway_geometry).length

In [ ]:
drainage_units_0_gdf = sjoin.loc[sjoin.groupby('drainage_unit_id')['overlap_length'].idxmax()]

In [ ]:
drainage_units_0_gdf = drainage_units_0_gdf.drop(columns=["drainage_unit_id_old", "waterway_geometry", "overlap_length"])

In [ ]:
drainage_units_0_gdf.to_file(Path(gdu.dir_results, "drainage_units_0_gdf.gpkg"))

In [ ]:
import imod

In [ ]:
gdu.drainage_units_0.data[0] = imod.prepare.rasterize(
    drainage_units_0_gdf.reset_index(drop=True), 
    column="drainage_unit_id",
    like=gdu.drainage_units_0[0]
)

In [ ]:
gdu.drainage_units_0 = gdu.drainage_units_0.fillna(-1)
gdu.drainage_units_0.attrs["_FillValue"] = -1

In [ ]:
netcdf_file_path = Path(gdu.dir_results, "drainage_units_0.nc")
encoding = {
    'drainage_units_0': {
        'dtype': 'float32',
        'zlib': True,
        'complevel': 9,
    },
}
gdu.drainage_units_0.to_netcdf(
    netcdf_file_path, 
    encoding=encoding
)